In [1]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [23]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub
import time

In [25]:
f = open("tok2.txt")
API_KEY = f.read()

llm = HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    huggingfacehub_api_token=API_KEY,
    model_kwargs={"temperature": 0.7, "max_length": 2000},
)


In [16]:
# Define prompt templates for the various prompting techniques.
template = PromptTemplate(
    input_variables=["schema","query_desc","prompt_technique"],
    template='''
    ### NLP DESCRIPTION OF THE SQL QUERY TO BE GENERATED:
    {query_desc}
    ### DATABASE SCHEMA OF THE MYSQL DATABASE THAT IS BEING WORKED ON:
    {schema}
    ### INSTRUCTION:
    The nlp description of an SQL query is given along with the schema of the database. Follow the instructions according to {prompt_technique} to generate a response.
    Do not add explanations, comments, or preamble. Output ONLY THE VALID SQL query.
    ### VALID SQL :
    '''
)



In [17]:
chain = LLMChain(llm=llm, prompt=template)

In [18]:

schema = '''
table name: listings
+-----------------------------+---------------+------+-----+---------+-------+
| Field                       | Type          | Null | Key | Default | Extra |
+-----------------------------+---------------+------+-----+---------+-------+
| listing_id                  | bigint        | NO   | PRI | NULL    |       |
| name                        | text          | YES  |     | NULL    |       |
| host_id                     | bigint        | YES  |     | NULL    |       |
| host_since                  | date          | YES  |     | NULL    |       |
| host_location               | text          | YES  |     | NULL    |       |
| host_response_time          | varchar(255)  | YES  |     | NULL    |       |
| host_response_rate          | float         | YES  |     | NULL    |       |
| host_acceptance_rate        | float         | YES  |     | NULL    |       |
| host_is_superhost           | tinyint(1)    | YES  |     | NULL    |       |
| host_total_listings_count   | int           | YES  |     | NULL    |       |
| host_has_profile_pic        | tinyint(1)    | YES  |     | NULL    |       |
| host_identity_verified      | tinyint(1)    | YES  |     | NULL    |       |
| neighbourhood               | varchar(255)  | YES  |     | NULL    |       |
| district                    | varchar(255)  | YES  |     | NULL    |       |
| city                        | varchar(255)  | YES  |     | NULL    |       |
| latitude                    | decimal(10,6) | YES  |     | NULL    |       |
| longitude                   | decimal(10,6) | YES  |     | NULL    |       |
| property_type               | varchar(255)  | YES  |     | NULL    |       |
| room_type                   | varchar(255)  | YES  |     | NULL    |       |
| accommodates                | int           | YES  |     | NULL    |       |
| bedrooms                    | int           | YES  |     | NULL    |       |
| amenities                   | json          | YES  |     | NULL    |       |
| price                       | decimal(10,2) | YES  |     | NULL    |       |
| minimum_nights              | int           | YES  |     | NULL    |       |
| maximum_nights              | int           | YES  |     | NULL    |       |
| review_scores_rating        | int           | YES  |     | NULL    |       |
| review_scores_accuracy      | int           | YES  |     | NULL    |       |
| review_scores_cleanliness   | int           | YES  |     | NULL    |       |
| review_scores_checkin       | int           | YES  |     | NULL    |       |
| review_scores_communication | int           | YES  |     | NULL    |       |
| review_scores_location      | int           | YES  |     | NULL    |       |
| review_scores_value         | int           | YES  |     | NULL    |       |
| instant_bookable            | tinyint(1)    | YES  |     | NULL    |       |
+-----------------------------+---------------+------+-----+---------+-------+

table name: reviews
+-------------+--------+------+-----+---------+-------+
| Field       | Type   | Null | Key | Default | Extra |
+-------------+--------+------+-----+---------+-------+
| review_id   | bigint | NO   | PRI | NULL    |       |
| listing_id  | bigint | YES  | MUL | NULL    |       |
| review_date | date   | YES  |     | NULL    |       |
| reviewer_id | bigint | YES  |     | NULL    |       |
+-------------+--------+------+-----+---------+-------+

'''

# zero shot prompting:
query_desc1 = "show me the top 5 most expensive Airbnb listings in New York"
query_desc2 = "Find all reviews for listings with a review score rating greater than 90."
query_desc3 = "Write a query to list all hosts who are superhosts and have more than 5 total listings."
query_desc4 = "Write a query to find all listings where the room_type is 'Entire home/apt' and the price is between $100 and $200 per night."
query_desc5 = "Write a query to return the listing_id, name, city, and price of the most expensive listing in each city."
query_desc6 = "Find the average review score rating of listings in each city."
query_desc7 = "Find all reviews from the last 6 months."
query_desc8 = "Write a query to find all listings that are instant_bookable and have a review_scores_cleanliness greater than 8."
query_desc9 = "Write a query to compute the average price of listings grouped by city, showing only cities where the average price is above $150."
query_desc10 = "Find listings in New York with the highest rating among those with more than 10 reviews."

prompt_technique1 = "given the query and the schema, generate the mysql query for the same."

In [19]:
response1 = chain.run(schema = schema, query_desc = query_desc1, prompt_technique = prompt_technique1 )
time.sleep(2)
response2 = chain.run(schema = schema, query_desc = query_desc2, prompt_technique = prompt_technique1 )
time.sleep(2)
response3 = chain.run(schema = schema, query_desc = query_desc3, prompt_technique = prompt_technique1 )
time.sleep(2)
response4 = chain.run(schema = schema, query_desc = query_desc4, prompt_technique = prompt_technique1 )
time.sleep(2)
response5 = chain.run(schema = schema, query_desc = query_desc5, prompt_technique = prompt_technique1 )
time.sleep(2)
response6 = chain.run(schema = schema, query_desc = query_desc6, prompt_technique = prompt_technique1 )
time.sleep(2)
response7 = chain.run(schema = schema, query_desc = query_desc7, prompt_technique = prompt_technique1 )
time.sleep(2)
response8 = chain.run(schema = schema, query_desc = query_desc8, prompt_technique = prompt_technique1 )
time.sleep(2)
response9 = chain.run(schema = schema, query_desc = query_desc9, prompt_technique = prompt_technique1 )
time.sleep(2)
response10 = chain.run(schema = schema, query_desc = query_desc10, prompt_technique = prompt_technique1 )
time.sleep(2)




print(response1)
print("-------------------------------------------------------------------------------------------------")
print(response2)
print("-------------------------------------------------------------------------------------------------")
print(response2)
print("-------------------------------------------------------------------------------------------------")
print(response3)
print("-------------------------------------------------------------------------------------------------")
print(response4)
print("-------------------------------------------------------------------------------------------------")
print(response5)
print("-------------------------------------------------------------------------------------------------")
print(response6)
print("-------------------------------------------------------------------------------------------------")
print(response7)
print("-------------------------------------------------------------------------------------------------")
print(response8)
print("-------------------------------------------------------------------------------------------------")
print(response9)
print("-------------------------------------------------------------------------------------------------")
print(response10)
print("-------------------------------------------------------------------------------------------------")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_


    ### NLP DESCRIPTION OF THE SQL QUERY TO BE GENERATED:
    show me the top 5 most expensive Airbnb listings in New York
    ### DATABASE SCHEMA OF THE MYSQL DATABASE THAT IS BEING WORKED ON:
    
table name: listings
+-----------------------------+---------------+------+-----+---------+-------+
| Field                       | Type          | Null | Key | Default | Extra |
+-----------------------------+---------------+------+-----+---------+-------+
| listing_id                  | bigint        | NO   | PRI | NULL    |       |
| name                        | text          | YES  |     | NULL    |       |
| host_id                     | bigint        | YES  |     | NULL    |       |
| host_since                  | date          | YES  |     | NULL    |       |
| host_location               | text          | YES  |     | NULL    |       |
| host_response_time          | varchar(255)  | YES  |     | NULL    |       |
| host_response_rate          | float         | YES  |     | NULL   

In [20]:
# Few-shot examples
examples = '''
-- Example 1:
-- NLP Query: Show me the top 10 most expensive listings in Los Angeles.
SELECT listing_id, name, price FROM listings WHERE city = 'Los Angeles' ORDER BY price DESC LIMIT 10;

-- Example 2:
-- NLP Query: Find all superhosts who have at least 10 total listings.
SELECT host_id FROM listings WHERE host_is_superhost = 1 AND host_total_listings_count >= 10;

-- Example 3:
-- NLP Query: List all listings in 'San Francisco' with a review score rating of at least 95.
SELECT listing_id, name, review_scores_rating FROM listings WHERE city = 'San Francisco' AND review_scores_rating >= 95;
'''

# Define the few-shot prompt technique
prompt_technique_few_shot = f"Use the given query and schema along with the following few-shot examples to generate an accurate MySQL query:\n{examples}"

# Generate responses for the same 10 queries using few-shot prompting
response_fs1 = chain.run(schema=schema, query_desc=query_desc1, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs2 = chain.run(schema=schema, query_desc=query_desc2, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs3 = chain.run(schema=schema, query_desc=query_desc3, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs4 = chain.run(schema=schema, query_desc=query_desc4, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs5 = chain.run(schema=schema, query_desc=query_desc5, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs6 = chain.run(schema=schema, query_desc=query_desc6, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs7 = chain.run(schema=schema, query_desc=query_desc7, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs8 = chain.run(schema=schema, query_desc=query_desc8, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs9 = chain.run(schema=schema, query_desc=query_desc9, prompt_technique=prompt_technique_few_shot)
time.sleep(2)
response_fs10 = chain.run(schema=schema, query_desc=query_desc10, prompt_technique=prompt_technique_few_shot)
time.sleep(2)

# Print the responses
print(response_fs1)
print("-------------------------------------------------------------------------------------------------")
print(response_fs2)
print("-------------------------------------------------------------------------------------------------")
print(response_fs3)
print("-------------------------------------------------------------------------------------------------")
print(response_fs4)
print("-------------------------------------------------------------------------------------------------")
print(response_fs5)
print("-------------------------------------------------------------------------------------------------")
print(response_fs6)
print("-------------------------------------------------------------------------------------------------")
print(response_fs7)
print("-------------------------------------------------------------------------------------------------")
print(response_fs8)
print("-------------------------------------------------------------------------------------------------")
print(response_fs9)
print("-------------------------------------------------------------------------------------------------")
print(response_fs10)
print("-------------------------------------------------------------------------------------------------")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_


    ### NLP DESCRIPTION OF THE SQL QUERY TO BE GENERATED:
    show me the top 5 most expensive Airbnb listings in New York
    ### DATABASE SCHEMA OF THE MYSQL DATABASE THAT IS BEING WORKED ON:
    
table name: listings
+-----------------------------+---------------+------+-----+---------+-------+
| Field                       | Type          | Null | Key | Default | Extra |
+-----------------------------+---------------+------+-----+---------+-------+
| listing_id                  | bigint        | NO   | PRI | NULL    |       |
| name                        | text          | YES  |     | NULL    |       |
| host_id                     | bigint        | YES  |     | NULL    |       |
| host_since                  | date          | YES  |     | NULL    |       |
| host_location               | text          | YES  |     | NULL    |       |
| host_response_time          | varchar(255)  | YES  |     | NULL    |       |
| host_response_rate          | float         | YES  |     | NULL   

In [21]:
# Chain-of-thought prompting

cot_reasoning = '''
### Step 1: Understand the Query
- Identify the key components in the natural language query (filters, sorting, aggregation, conditions).

### Step 2: Map to Schema
- Match relevant fields in the database schema to the query's requirements.

### Step 3: Construct SQL Query
- Formulate the correct SQL query based on identified fields and conditions.
'''

# Define the Chain-of-Thought prompt technique
prompt_technique_cot = f"Follow these step-by-step reasoning steps before generating the SQL query:\n{cot_reasoning}"

# Generate responses for the same 10 queries using Chain-of-Thought prompting
response_cot1 = chain.run(schema=schema, query_desc=query_desc1, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot2 = chain.run(schema=schema, query_desc=query_desc2, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot3 = chain.run(schema=schema, query_desc=query_desc3, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot4 = chain.run(schema=schema, query_desc=query_desc4, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot5 = chain.run(schema=schema, query_desc=query_desc5, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot6 = chain.run(schema=schema, query_desc=query_desc6, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot7 = chain.run(schema=schema, query_desc=query_desc7, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot8 = chain.run(schema=schema, query_desc=query_desc8, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot9 = chain.run(schema=schema, query_desc=query_desc9, prompt_technique=prompt_technique_cot)
time.sleep(2)
response_cot10 = chain.run(schema=schema, query_desc=query_desc10, prompt_technique=prompt_technique_cot)
time.sleep(2)

# Print the responses
print(response_cot1)
print("-------------------------------------------------------------------------------------------------")
print(response_cot2)
print("-------------------------------------------------------------------------------------------------")
print(response_cot3)
print("-------------------------------------------------------------------------------------------------")
print(response_cot4)
print("-------------------------------------------------------------------------------------------------")
print(response_cot5)
print("-------------------------------------------------------------------------------------------------")
print(response_cot6)
print("-------------------------------------------------------------------------------------------------")
print(response_cot7)
print("-------------------------------------------------------------------------------------------------")
print(response_cot8)
print("-------------------------------------------------------------------------------------------------")
print(response_cot9)
print("-------------------------------------------------------------------------------------------------")
print(response_cot10)
print("-------------------------------------------------------------------------------------------------")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_


    ### NLP DESCRIPTION OF THE SQL QUERY TO BE GENERATED:
    show me the top 5 most expensive Airbnb listings in New York
    ### DATABASE SCHEMA OF THE MYSQL DATABASE THAT IS BEING WORKED ON:
    
table name: listings
+-----------------------------+---------------+------+-----+---------+-------+
| Field                       | Type          | Null | Key | Default | Extra |
+-----------------------------+---------------+------+-----+---------+-------+
| listing_id                  | bigint        | NO   | PRI | NULL    |       |
| name                        | text          | YES  |     | NULL    |       |
| host_id                     | bigint        | YES  |     | NULL    |       |
| host_since                  | date          | YES  |     | NULL    |       |
| host_location               | text          | YES  |     | NULL    |       |
| host_response_time          | varchar(255)  | YES  |     | NULL    |       |
| host_response_rate          | float         | YES  |     | NULL   

In [22]:
# Self-Consistency: Generate multiple candidate SQL queries
self_consistency_prompt = '''
### Step 1: Generate Multiple SQL Queries
- Create at least three different SQL queries using slightly varied reasoning paths.

### Step 2: Compare Queries
- Check for common patterns and consistency across queries.

### Step 3: Select the Best Query
- Use majority voting or confidence in structure to determine the most reliable SQL output.
'''

# Define the Self-Consistency prompt technique
prompt_technique_self_consistency = f"Follow these self-consistency steps before generating the final SQL query:\n{self_consistency_prompt}"

# Generate responses for the same 10 queries using Self-Consistency prompting
response_sc1 = chain.run(schema=schema, query_desc=query_desc1, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc2 = chain.run(schema=schema, query_desc=query_desc2, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc3 = chain.run(schema=schema, query_desc=query_desc3, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc4 = chain.run(schema=schema, query_desc=query_desc4, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc5 = chain.run(schema=schema, query_desc=query_desc5, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc6 = chain.run(schema=schema, query_desc=query_desc6, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc7 = chain.run(schema=schema, query_desc=query_desc7, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc8 = chain.run(schema=schema, query_desc=query_desc8, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc9 = chain.run(schema=schema, query_desc=query_desc9, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)
response_sc10 = chain.run(schema=schema, query_desc=query_desc10, prompt_technique=prompt_technique_self_consistency)
time.sleep(2)

# Print the responses
print(response_sc1)
print("-------------------------------------------------------------------------------------------------")
print(response_sc2)
print("-------------------------------------------------------------------------------------------------")
print(response_sc3)
print("-------------------------------------------------------------------------------------------------")
print(response_sc4)
print("-------------------------------------------------------------------------------------------------")
print(response_sc5)
print("-------------------------------------------------------------------------------------------------")
print(response_sc6)
print("-------------------------------------------------------------------------------------------------")
print(response_sc7)
print("-------------------------------------------------------------------------------------------------")
print(response_sc8)
print("-------------------------------------------------------------------------------------------------")
print(response_sc9)
print("-------------------------------------------------------------------------------------------------")
print(response_sc10)
print("-------------------------------------------------------------------------------------------------")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_


    ### NLP DESCRIPTION OF THE SQL QUERY TO BE GENERATED:
    show me the top 5 most expensive Airbnb listings in New York
    ### DATABASE SCHEMA OF THE MYSQL DATABASE THAT IS BEING WORKED ON:
    
table name: listings
+-----------------------------+---------------+------+-----+---------+-------+
| Field                       | Type          | Null | Key | Default | Extra |
+-----------------------------+---------------+------+-----+---------+-------+
| listing_id                  | bigint        | NO   | PRI | NULL    |       |
| name                        | text          | YES  |     | NULL    |       |
| host_id                     | bigint        | YES  |     | NULL    |       |
| host_since                  | date          | YES  |     | NULL    |       |
| host_location               | text          | YES  |     | NULL    |       |
| host_response_time          | varchar(255)  | YES  |     | NULL    |       |
| host_response_rate          | float         | YES  |     | NULL   